In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

def forecast_aqi(file_path, seq_length=24, future_steps=8):
    # Load and preprocess the dataset
    df = pd.read_csv(file_path)
    df.set_index('Date_Start Time', inplace=True)
    df.drop(['Sr.NO', 'End Time'], axis=1, inplace=True)

    # Ensure AQI columns are float
    aqi_columns = ['AQI_PM2.5', 'AQI_PM10', 'AQI_NO2', 'AQI_CO', 'AQI_SO2']
    for col in aqi_columns:
        df[col] = df[col].astype(float)

    # Normalize the data using MinMaxScaler
    scalers = {}
    scaled_data = {}
    for col in aqi_columns:
        scaler = MinMaxScaler()
        scaled_data[col] = scaler.fit_transform(df[[col]])
        scalers[col] = scaler

    # Function to forecast future AQI values
    def forecast_future(model, data, seq_length, future_steps):
        future_predictions = []
        current_seq = data[-seq_length:]  # Start from the last available sequence
        
        for _ in range(future_steps):
            pred = model.predict(current_seq[np.newaxis, :, :])  # Predict next step
            future_predictions.append(pred[0])
            current_seq = np.append(current_seq[1:], pred, axis=0)  # Update the sequence
        
        return np.array(future_predictions)

    # Load pre-trained models and forecast the next 'n' hours
    future_predictions = {}
    for col in aqi_columns:
        model = load_model(f'{col}_LSTM_model.h5')  # Load pre-trained LSTM model
        future_scaled_predictions = forecast_future(model, scaled_data[col], seq_length, future_steps)
        # Inverse transform to get the actual AQI values
        future_predictions[col] = scalers[col].inverse_transform(future_scaled_predictions).flatten()

    # Convert future predictions to a DataFrame with a proper timestamp index
    future_df = pd.DataFrame(future_predictions, index=pd.date_range(start=df.index[-1], periods=future_steps+1, freq='h')[1:])
    return future_df

# File path of the dataset
file_path = '../ARIF/AQI_Weather_Data.csv'

# Call the forecast function to get future AQI predictions
future_df = forecast_aqi(file_path)

# Display the predicted values
print(future_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
                     AQI_PM2.5    AQI_PM10    AQI_NO2     AQI_CO    AQI_SO2
2023-04-01 00:00:00  42.472965  102.229370  49.327312  31.932623  28.024145
2023-04-01 01:00:00  43.372379  102.931725  49.198601  33.566483  28.203175
2023-04-01 02:00:00  44.211086  103.279778  49.051525  34.603947  28.438957
2023-04-01 03:00:00  44.921944  103.427887  48.899712  35.322994  28.714672
2023-04-01 04:00:00  45.542187  103.380775  48.747456  36.013939  29.024403
2023-04-01 05:00:00  46.092628  103.171753  48.596684  36.850441  29.361322
2023-04-01 06:00:00  46.587616  102.838280  48.447876  37.833908  29.719286
2023-04-01 07:00:00  47.038719  102.413216  48.300694  38.829639  30.093285


In [19]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import pandas as pd

def forecast_aqi_from_current(current_aqi, seq_length=24, future_steps=8):
    """
    This function forecasts future AQI values based on the current AQI data passed as input.

    :param current_aqi: A dictionary with current AQI values, e.g.:
                        {'AQI_PM2.5': 45.0, 'AQI_PM10': 60.0, 'AQI_NO2': 20.0, 'AQI_CO': 1.5, 'AQI_SO2': 10.0}
    :param seq_length: Number of previous steps to use for the LSTM model (default: 24 hours)
    :param future_steps: Number of future hours to predict (default: 8 hours)
    :return: A DataFrame with the predicted future AQI values for the next 'future_steps' hours.
    """
    
    aqi_columns = ['AQI_PM2.5', 'AQI_PM10', 'AQI_NO2', 'AQI_CO', 'AQI_SO2']

    # Normalize current AQI values
    scalers = {}
    scaled_current_aqi = {}
    
    for col in aqi_columns:
        scaler = MinMaxScaler()
        # Fit the scaler on a range, assuming typical AQI value ranges (e.g., 0 to 500 for PM2.5 and PM10, etc.)
        scaler.fit(np.array([0, 500]).reshape(-1, 1))
        scaled_current_aqi[col] = scaler.transform(np.array(current_aqi[col]).reshape(-1, 1))
        scalers[col] = scaler

    # Function to forecast future AQI values
    def forecast_future(model, data, seq_length, future_steps):
        future_predictions = []
        current_seq = np.array(data[-seq_length:])  # Start from the last available sequence
        
        for _ in range(future_steps):
            pred = model.predict(current_seq[np.newaxis, :, :])  # Predict next step
            future_predictions.append(pred[0])
            current_seq = np.append(current_seq[1:], pred, axis=0)  # Update the sequence

        return np.array(future_predictions)

    # Load pre-trained models and forecast the next 'n' hours
    future_predictions = {}
    
    for col in aqi_columns:
        model = load_model(f'{col}_LSTM_model.h5')  # Load pre-trained LSTM model
        
        # Assuming current AQI values serve as the last known sequence
        future_scaled_predictions = forecast_future(model, scaled_current_aqi[col], seq_length, future_steps)
        
        # Inverse transform to get the actual AQI values
        future_predictions[col] = scalers[col].inverse_transform(future_scaled_predictions).flatten()

    # Convert future predictions to a DataFrame with a proper timestamp index
    future_df = pd.DataFrame(future_predictions, index=pd.date_range(start=pd.Timestamp.now(), periods=future_steps, freq='H'))
    return future_df

# Example Usage:
current_aqi = {
    'AQI_PM2.5': 45.0,
    'AQI_PM10': 60.0,
    'AQI_NO2': 20.0,
    'AQI_CO': 1.5,
    'AQI_SO2': 10.0
}

# Call the function to get future AQI predictions
future_df = forecast_aqi_from_current(current_aqi)

# Display the predicted future AQI values
print(future_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
                             AQI_PM2.5    AQI_PM10    AQI_NO2      AQI_CO  \
2024-08-19 00:05:49.847626  -18.761978   33.683098  23.624792  -45.963631   
2024-08-19 01:05:49.847626 -110.532150   -1.685523  26.746155 -103.369652   
2024-08-19 02:05:49.847626 -215.434692  -46.177261  29.455200 -166.950134   
2024-08-19 03:05:49.847626 -302.007690  -97.323051  31.822323 -230.052765   
2024-08-19 04:05:49.847626 -352.337891 -149.779343  33.902912 -285.457611   
2024-08-19 05:05:49.847626 -374.706970 -196.995850  35.741009 -328.595825   
2024-08-19 06:05:49.847626 -383.282349 -234.247513  37.372246 -358.900909   
2024-08-19 07:05:49.847626 -386.366547 -260.417633  38.825699 -378.597504   

         

/var/folders/0b/tbcb989n38q5j2gz3d4zcx680000gn/T/ipykernel_68398/3387477458.py:55: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_df = pd.DataFrame(future_predictions, index=pd.date_range(start=pd.Timestamp.now(), periods=future_steps, freq='H'))


In [20]:
for row in future_df.iterrows():
    print(row)

(Timestamp('2024-08-19 00:05:49.847626'), AQI_PM2.5   -18.761978
AQI_PM10     33.683098
AQI_NO2      23.624792
AQI_CO      -45.963631
AQI_SO2     -47.008671
Name: 2024-08-19 00:05:49.847626, dtype: float32)
(Timestamp('2024-08-19 01:05:49.847626'), AQI_PM2.5   -110.532150
AQI_PM10      -1.685523
AQI_NO2       26.746155
AQI_CO      -103.369652
AQI_SO2     -122.318588
Name: 2024-08-19 01:05:49.847626, dtype: float32)
(Timestamp('2024-08-19 02:05:49.847626'), AQI_PM2.5   -215.434692
AQI_PM10     -46.177261
AQI_NO2       29.455200
AQI_CO      -166.950134
AQI_SO2     -210.089890
Name: 2024-08-19 02:05:49.847626, dtype: float32)
(Timestamp('2024-08-19 03:05:49.847626'), AQI_PM2.5   -302.007690
AQI_PM10     -97.323051
AQI_NO2       31.822323
AQI_CO      -230.052765
AQI_SO2     -296.551971
Name: 2024-08-19 03:05:49.847626, dtype: float32)
(Timestamp('2024-08-19 04:05:49.847626'), AQI_PM2.5   -352.337891
AQI_PM10    -149.779343
AQI_NO2       33.902912
AQI_CO      -285.457611
AQI_SO2     -366.71